In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/20 08:34:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [4]:
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [6]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [7]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [8]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [9]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [10]:
!wc -l head.csv

1001 head.csv


In [11]:
import pandas as pd
pd.__version__

'2.2.2'

In [12]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.4.4
      /_/
                        
Using Scala version 2.12.17, OpenJDK 64-Bit Server VM, 11.0.2
Branch HEAD
Compiled by user ubuntu on 2024-10-21T02:09:45Z
Revision 6729992c76fc59ab07f63f97a9858691274447d0
Url https://github.com/apache/spark
Type --help for more information.


In [13]:
df_pandas = pd.read_csv('head.csv')

In [14]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [15]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

#### Integer - 4 bytes Long - 8 bytes

In [16]:
from pyspark.sql import types

In [17]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [18]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [19]:
df = df.repartition(24)

In [21]:
# df.write.parquet('fhvhv/2021/01/')

In [22]:
df = spark.read.parquet('fhvhv/2021/01/')

In [23]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [24]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [25]:
df.select('*') \
  .filter(df.hvfhs_license_num == 'HV0003') \
  .show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02882|2021-01-05 22:14:07|2021-01-05 22:32:28|         189|         107|   null|
|           HV0003|              B02867|2021-01-02 17:59:55|2021-01-02 18:10:39|          88|         137|   null|
|           HV0003|              B02872|2021-01-02 23:57:54|2021-01-03 00:15:48|         238|         224|   null|
|           HV0003|              B02875|2021-01-06 15:53:13|2021-01-06 16:07:07|         169|         208|   null|
|           HV0003|              B02867|2021-01-07 07:35:24|2021-01-07 07:55:49|          75|          88|   null|
|           HV0003|              B02764|2021-01-07 08:45:12|2021-01-07 08:51:17|

In [26]:
from pyspark.sql import functions as F

In [42]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-03|  2021-01-03|         255|          34|
| 2021-01-05|  2021-01-05|         189|         107|
| 2021-01-02|  2021-01-02|          88|         137|
| 2021-01-02|  2021-01-03|         238|         224|
| 2021-01-06|  2021-01-06|         169|         208|
| 2021-01-07|  2021-01-07|          75|          88|
| 2021-01-07|  2021-01-07|         210|         210|
| 2021-01-02|  2021-01-02|         243|          69|
| 2021-01-04|  2021-01-04|         250|         213|
| 2021-01-03|  2021-01-03|          87|          79|
| 2021-01-03|  2021-01-03|          68|         181|
| 2021-01-04|  2021-01-04|          95|         236|
| 2021-01-02|  2021-01-02|         262|         236|
| 2021-01-04|  2021-01-04|         225|         233|
| 2021-01-06|  2021-01-06|         237|          83|
| 2021-01-05|  2021-01-05|         231|       

In [47]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}' 

In [48]:
crazy_stuff('B02884')

's/b44'

In [51]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType = types.StringType())

In [53]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id','pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-03|  2021-01-03|         255|          34|
|  e/b42| 2021-01-05|  2021-01-05|         189|         107|
|  e/b33| 2021-01-02|  2021-01-02|          88|         137|
|  e/b38| 2021-01-02|  2021-01-03|         238|         224|
|  e/b3b| 2021-01-06|  2021-01-06|         169|         208|
|  e/b33| 2021-01-07|  2021-01-07|          75|          88|
|  e/acc| 2021-01-07|  2021-01-07|         210|         210|
|  e/acc| 2021-01-02|  2021-01-02|         243|          69|
|  e/b35| 2021-01-04|  2021-01-04|         250|         213|
|  s/b3d| 2021-01-03|  2021-01-03|          87|          79|
|  e/a39| 2021-01-03|  2021-01-03|          68|         181|
|  s/acd| 2021-01-04|  2021-01-04|          95|         236|
|  s/b13| 2021-01-02|  2021-01-02|         262|         236|
|  e/9ce| 2021-01-04|  2